In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/1016version.csv')

In [109]:
positions = ['SUPPORT', 'ADC', 'MIDDLE', 'JUNGLE', 'TOP']
t1_fields = []

for position in range(1, 6):
    field = f't1p{position}_champId'
    t1_fields.append(field)

t1 = df[t1_fields].copy()
t1['t1_win'] = df['t1_win']
t1 = t1.melt(id_vars=['t1_win'], value_vars=t1_fields[:], var_name='position', value_name='champId')
t1['position'] = t1['position'].replace(dict(zip(t1_fields, positions)))
t1 = t1.groupby(['position', 'champId']).agg({'t1_win': ['count', 'sum', 'mean']})
t1.columns = ['appearances', 'wins', 'winrate']
t1 = t1.reset_index()


In [111]:
t2_fields = []

for position in range(1, 6):
    field = f't2p{position}_champId'
    t2_fields.append(field)

t2 = df[t2_fields].copy()
t2['t2_win'] = df['t1_win'].replace({0: 1, 1: 0})
t2 = t2.melt(id_vars=['t2_win'], value_vars=t2_fields[:], var_name='position', value_name='champId')
t2['position'] = t2['position'].replace(dict(zip(t2_fields, positions)))
t2 = t2.groupby(['position', 'champId']).agg({'t2_win': ['count', 'sum', 'mean']})
t2.columns = ['appearances', 'wins', 'winrate']
t2 = t2.reset_index()

In [121]:
t2

,position,champId,appearances,wins,winrate
0,ADC,1,8,7,0.875000
1,ADC,2,14,9,0.642857
2,ADC,3,12,8,0.666667
3,ADC,7,1,0,0.000000
4,ADC,8,15,6,0.400000
...,...,...,...,...,...
590,TOP,518,288,142,0.493056
591,TOP,555,37,23,0.621622
592,TOP,777,659,319,0.484067
593,TOP,875,947,458,0.483633
